In [3]:
pwd

'/content'

In [4]:
# mkdir gotbooks

In [6]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 78.9 MB/s eta 0:00:00


In [7]:
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api
import gensim

In [ ]:
# google news corpus is used for training containing 3 billion words
# 300 dimensional vectors for 3 million words and phrases

In [8]:
word_emb = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [9]:
word_emb.get_index('car')

385

In [10]:
word_emb.get_vector('car').shape

(300,)

In [11]:
word_emb['man']

array([ 0.32617188,  0.13085938,  0.03466797, -0.08300781,  0.08984375,
       -0.04125977, -0.19824219,  0.00689697,  0.14355469,  0.0019455 ,
        0.02880859, -0.25      , -0.08398438, -0.15136719, -0.10205078,
        0.04077148, -0.09765625,  0.05932617,  0.02978516, -0.10058594,
       -0.13085938,  0.001297  ,  0.02612305, -0.27148438,  0.06396484,
       -0.19140625, -0.078125  ,  0.25976562,  0.375     , -0.04541016,
        0.16210938,  0.13671875, -0.06396484, -0.02062988, -0.09667969,
        0.25390625,  0.24804688, -0.12695312,  0.07177734,  0.3203125 ,
        0.03149414, -0.03857422,  0.21191406, -0.00811768,  0.22265625,
       -0.13476562, -0.07617188,  0.01049805, -0.05175781,  0.03808594,
       -0.13378906,  0.125     ,  0.0559082 , -0.18261719,  0.08154297,
       -0.08447266, -0.07763672, -0.04345703,  0.08105469, -0.01092529,
        0.17480469,  0.30664062, -0.04321289, -0.01416016,  0.09082031,
       -0.00927734, -0.03442383, -0.11523438,  0.12451172, -0.02

In [12]:
word_emb.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824871301651001),
 ('teenager', 0.6586930155754089),
 ('teenage_girl', 0.6147903203964233),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.571636438369751),
 ('robber', 0.5585119128227234),
 ('Robbery_suspect', 0.5584409832954407),
 ('teen_ager', 0.5549196600914001),
 ('men', 0.5489763021469116)]

In [14]:
word_emb.most_similar('animal')

[('animals', 0.7875037789344788),
 ('Animal', 0.7282052040100098),
 ('animal_welfare', 0.6635975241661072),
 ('dog', 0.6438013315200806),
 ('pet', 0.6338438987731934),
 ('cats', 0.6126771569252014),
 ('ani_mal', 0.6077603101730347),
 ('Animals', 0.6017201542854309),
 ('dogs', 0.5963991284370422),
 ('feline', 0.5933956503868103)]

In [13]:
word_emb.most_similar('women')

[('men', 0.767493724822998),
 ('Women', 0.7283449769020081),
 ('womens', 0.6786180734634399),
 ('girls', 0.6339040398597717),
 ('females', 0.6240420341491699),
 ('mothers', 0.6050933599472046),
 ('ladies', 0.5865179300308228),
 ('husbands', 0.5705342292785645),
 ('transwomen', 0.5697940587997437),
 ('Men', 0.5693342089653015)]

In [15]:
# so it contains vectors of words

In [16]:
# types of word to vec => 1. CBOW   2. Skip-gram

# Custom word2vec embeddings

In [17]:
import nltk
import pandas as pd
import numpy as np
import os

In [18]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
# creating a list to keep the entire data
# the data used is game of thrones books (txt files) from kaggle
# link -> https://www.kaggle.com/datasets/khulasasndh/game-of-thrones-books
story = []

In [19]:
# list of text files

for file in os.listdir("./gotbooks"):
    print(file)

003ssb.txt
001ssb.txt
005ssb.txt
004ssb.txt
002ssb.txt


In [21]:
folder_path = "./gotbooks"

In [22]:
# function to tokenize, preprocess and store all the files in empty list ->  story

def preprocessor(folder_path_of_files):
    story= []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        with open (file_path, encoding= 'unicode_escape') as f:
            corpus = f.read()
        raw_sent = sent_tokenize(corpus)
        # preprocess using simple_preprocess and store in story list
        for sent in raw_sent:
            story.append(simple_preprocess(sent))

    return story

In [26]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [28]:
story = preprocessor(folder_path)

In [29]:
story[0]

['storm',
 'of',
 'swords',
 'book',
 'three',
 'of',
 'song',
 'of',
 'ice',
 'and',
 'fire',
 'by',
 'george']

In [32]:
len(story)

145020

In [33]:
story[34]

['small', 'paul', 'blinked', 'his', 'black', 'little', 'eyes']

#### Build the custom word2vec

In [34]:
# using CBOW
custom_model = gensim.models.Word2Vec(window= 10, min_count= 5,
                                      vector_size= 150,
                                      workers = 12, epochs= 5)

# min_count => dont capture the senteces with less than 5 words
# vector_size => dimensions
# by default it uses cbow (sg = 0), if want to use skip-gram set parameter sg = 1
# to use more cpu cores during training => workers = no_of_cpu_cores

In [36]:
# if u want to check how many cpu cores u have
import os
print(os.cpu_count())

2


In [37]:
# build the vocab
custom_model.build_vocab(story)

In [38]:
custom_model.corpus_count
# no. of sentences we have

145020

In [39]:
# no of unique words
custom_model.corpus_total_words

1725638

In [40]:
print(custom_model.epochs)

5


In [41]:
# train the model now
custom_model.train(story,
                   total_examples=custom_model.corpus_count,
                   epochs = custom_model.epochs)

(6482433, 8628190)

In [42]:
# check

custom_model.wv.most_similar('stark')

[('arryn', 0.7707759737968445),
 ('winterfell', 0.7393332719802856),
 ('brandon', 0.7314446568489075),
 ('tully', 0.7079401612281799),
 ('greyjoy', 0.6987162828445435),
 ('robb', 0.698350727558136),
 ('bastard', 0.6865304112434387),
 ('eddard', 0.6860319972038269),
 ('benjen', 0.6809831261634827),
 ('murdered', 0.6607222557067871)]

In [43]:
custom_model.wv.most_similar('dwarf')

[('hound', 0.7182908058166504),
 ('imp', 0.7100985646247864),
 ('lad', 0.6945008635520935),
 ('reek', 0.6888728737831116),
 ('tyrion', 0.6768685579299927),
 ('wench', 0.6670268774032593),
 ('griff', 0.6612421870231628),
 ('answer', 0.6577406525611877),
 ('question', 0.6552461385726929),
 ('joff', 0.653850793838501)]

In [45]:
custom_model.wv.most_similar('king')

[('prince', 0.7010721564292908),
 ('baratheon', 0.6657240390777588),
 ('council', 0.6461156606674194),
 ('realm', 0.6439169049263),
 ('throne', 0.6067066192626953),
 ('tourney', 0.600085973739624),
 ('conqueror', 0.580512285232544),
 ('targaryen', 0.5604947209358215),
 ('lannisters', 0.5572969317436218),
 ('aegon', 0.5507369637489319)]

In [46]:
custom_model.wv.most_similar('dragon')

[('star', 0.6029009819030762),
 ('world', 0.5911185145378113),
 ('kingdoms', 0.5754508972167969),
 ('westeros', 0.5718582272529602),
 ('conqueror', 0.5657942891120911),
 ('dragons', 0.5578041076660156),
 ('throne', 0.5560122132301331),
 ('dothrak', 0.5392627120018005),
 ('god', 0.5373312830924988),
 ('sea', 0.5341046452522278)]

In [ ]:
custom_model.wv['king'].shape

(150,)

In [ ]:
# Note: From the data we have not removed the stopswords which we should have